In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# add parent folder path where lib folder is
import sys
if ".." not in sys.path:import sys; sys.path.insert(0, '..') 

In [ ]:
# nonparametric tests for one performance measure (e.g., AUC)
from P_HAK import run_friedman, stac_multi, ap2h0

In [ ]:
alpha = 0.05   # Set this to change the default signifance level

#### Read in our data

In [ ]:
# In the csv, the columns are classifiers and the rows are datasets 
full_df = pd.read_csv("../datasets/gmeans.csv", index_col=0)

In [ ]:
# All_vs_One: list of "control" classifiers
#avo = []
avo = ['RF']
#avo = ['RF','XGB']

In [ ]:
if len(avo) == 0:
    oname = "All_Models"
    df = full_df
else:
    baseclf = tuple(avo)
    oname=""
    for x in range(len(avo)):
        oname += avo[x] + "_"
    oname += "Models"
    df = full_df.loc[:, full_df.columns.str.startswith(baseclf)]

#### Friedman test 
Checks if there is a significant difference in performance for any classifier<br>
If we reject H0 (no difference), we use the post-hoc test to find out which differences are significant.

In [ ]:
reject, rptstr, rankings = run_friedman(df)
print(oname,":",rptstr)

# continue only if H0 was rejected
if not reject:
    raise Exception("Accepted H0 for Freidman Test")

#### All vs. All Tests
Compare every classifier to every other one, using the rankings ('pivotal values')  returned from the Freidman test.<br>
General case shows p_values adjusted for multiple tests using a range of methods, Nemenyi and Shaffer show p_values adjusted using similar methods.Note that for technical reasons, the Schaffer method should not be used for more than 18 classifiers.<br>
The dataframe of adjusted p_values can be quickly converted to show if the Null Hypothesis (H0: No significant difference) should be accepted (True) or rejected (False).

In [ ]:
gen_pvals_df = stac_multi(rankings)
gen_pvals_df

In [ ]:
gen_ho_df = ap2h0(gen_pvals_df)
gen_ho_df

In [ ]:
nym_ap_df=stac_multi(rankings,nmyi=True)
nym_ap_df

#### One vs All (Control vs Treatment) 
In some cases, we do not care about all pairwise comparisons as we only propose a single method, or just need to compare to a baseline method. In this case we designate a control method, and compare all others to it.<br>
For statistical reasons the Nemenyi test should not be used for this.

In [ ]:
xgb_ap_df = stac_multi(rankings,control='RF')
xgb_ap_df

In [ ]:
xgb_ho_df = ap2h0(xgb_ap_df)
xgb_ho_df

In [ ]:
shaf_ap_xgb_df = stac_multi(rankings,shaf=True,control='RF')
shaf_ap_xgb_df